**实验目标：**

通过本实验，你将深入了解和实践说话人识别技术，并掌握利用声音特征进行有效说话人识别的基本方法，了解不同特征和模型对识别准确率的影响。

实验的核心目标是使用TIMIT数据集来训练一个说话人识别系统，涵盖数据预处理、特征提取、模型训练和评估等关键步骤。


**实验方法：**

**1. 数据预处理和划分(可选)：**
  - 为了方便大家，我们提供了划分好的TIMIT数据集结构，当然你也可以根据训练结果自行划分该原数据集。
  - 原数据集下载地址：https://drive.google.com/file/d/180mSIiXN9RVDV2Xn1xcWNkMRm5J5MjN4/view?usp=sharing
  - 我们排除了SA的两个方言句子，并在剩余的8个句子中选取了SX的5个句子和SI的1个句子作为训练集，SI的另外2个句子作为测试集。
  
**2. 特征提取：**
  - 学习并实现包括但不限于MFCC特征等特征的提取，探索声音信号的频率和时间特性。
  - 鼓励尝试和比较其他特征提取方法，例如LPCC或声谱图特征，以理解不同特征对识别性能的影响。
  
**3. 模型选择和训练：**
  - 探索并选择适合的分类器和模型进行说话人识别，如GMM、Softmax分类器或深度学习模型。
  - 实现模型训练流程，使用训练集数据训练模型。
  
**4. 评估和分析：**
  - 使用准确率作为主要的评价指标在测试集上评估模型性能。
  - 对比不同特征和模型的性能，分析其对说话人识别准确率的影响。
  - 可视化不同模型的识别结果和错误率，讨论可能的改进方法。

**实验要求：**
  - 1.选择并实现至少一种特征的提取，并鼓励尝试其他特征提取方法。
  - 2.选择并实现至少一种分类器或模型进行说话人识别，并使用准确率评估指标评估其性能。
  - 3.通过实验对比、分析和可视化，撰写详细的实验报告，包括实验目的、实验方法、结果分析和结论。
  - 4.实验报告应以清晰、逻辑性强的形式呈现，图表和结果应清楚明了。

## 1. 实验准备

In [2]:
## 导入必要的库
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
import os
import soundfile as sf
import librosa
from sklearn.preprocessing import StandardScaler
from sklearn.mixture import GaussianMixture
from sklearn.metrics import accuracy_score

# 可以根据需要导入其他库，比如librosa用于音频处理

D:\Anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
D:\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
D:\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
D:\Anaconda3\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


## 2. 数据预处理(加载数据集)

In [3]:
TrainDir = "Dataset\TRAIN"
TestDir = "Dataset\TEST"


## 请在这里写代码加载我们划分好的TIMIT训练集和测试集

def load_timit_dataset(base_path):
    speakers_data = {}
    for root, dirs, files in os.walk(base_path):
        for file in files:
            if file.endswith(".wav"):
                speaker_id = root.split(os.sep)[-1]
                file_path = os.path.join(root, file)
                data, samplerate = sf.read(file_path)
                if speaker_id not in speakers_data:
                    speakers_data[speaker_id] = []
                speakers_data[speaker_id].append((data, samplerate))
    return speakers_data


train_data = load_timit_dataset(TrainDir)
test_data = load_timit_dataset(TestDir)

## 3. 特征提取

In [6]:
## 请编写或使用库函数提取MFCC等音频特征
def extract_features(audio_data, samplerate):
    mfccs = librosa.feature.mfcc(y=audio_data, sr=samplerate, n_mfcc=13)
    return mfccs


def prepare_dataset(speakers_data):
    features = []
    labels = []
    for speaker_id, audios in speakers_data.items():
        for audio, rate in audios:
            mfcc = extract_features(audio, rate)
            features.append(mfcc)
            labels.append(speaker_id)
    return features, np.array(labels)  # 这里将 labels 转换为 numpy 数组


train_data_features, train_data_labels = prepare_dataset(train_data)
test_data_features, test_data_labels = prepare_dataset(test_data)


[[-7.71472487e+02 -7.13510832e+02 -6.78341201e+02 ... -6.68288882e+02
  -6.92413130e+02 -7.32619436e+02]
 [ 1.68135663e+01  2.32430109e+01  2.65236342e+01 ...  3.81183607e+01
   3.26592152e+01  3.47468431e+01]
 [ 1.95008782e+01 -1.82944429e+00 -2.61476901e+01 ... -1.91550783e+01
  -1.14221889e+01  5.88391911e-02]
 ...
 [ 7.67943610e+00 -1.07766150e+00 -2.72704264e+00 ... -6.72544537e+00
  -2.12274240e+00  1.49642342e+00]
 [ 5.13097026e+00  1.47870350e+00 -1.76497998e+00 ... -4.47882995e+00
  -4.50797310e+00 -5.94221878e+00]
 [ 4.30375940e+00  5.85914219e+00  1.13321518e+01 ...  1.14590814e+01
   7.14151428e+00  6.09138507e+00]]


## 4. 模型选择和训练

In [9]:
## 在这部分，你可以选择不同的分类器和模型如GMM模型来进行实验
def flatten_features(features):
    # 将特征从三维数组转换为二维数组，以便可以用于GMM
    flattened_features = []
    for feature in features:
        # 使用均值跨时间序列来简化数据，可以尝试其他方法如最大值、中位数等
        flattened_features.append(np.mean(feature, axis=1))
    return np.vstack(flattened_features)


# 数据扁平化
X_train = flatten_features(train_data_features)
X_test = flatten_features(test_data_features)

# 标准化特征
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# 确定GMM组件的数量
n_components = len(np.unique(train_data_labels))

# 训练GMM
gmm = GaussianMixture(n_components=n_components, covariance_type='diag', max_iter=200, random_state=0)
gmm.fit(X_train)

GaussianMixture(covariance_type='diag', max_iter=200, n_components=462,
                random_state=0)

## 5. 评价指标(准确率Accuracy)

In [10]:
## 请编写代码或使用库函数accuracy_score计算测试集上的准确率Accuracy
# 对训练数据进行聚类
train_labels_gmm = gmm.predict(X_train)

# 创建从GMM组件到实际标签的映射
from scipy.stats import mode

labels_mapping = {}
for i in range(n_components):
    mask = (train_labels_gmm == i)
    # 为每个组件找出最常见的标签
    if np.any(mask):
        labels_mapping[i] = mode(train_data_labels[mask])[0][0]

# 使用映射进行测试数据的预测
test_labels_gmm = gmm.predict(X_test)
predicted_labels = [labels_mapping[label] for label in test_labels_gmm]

# 计算准确率
accuracy = accuracy_score(test_data_labels, predicted_labels)
print(f"Test Accuracy: {accuracy:.2f}")

Test Accuracy: 0.20


C:\Users\Edgar\AppData\Local\Temp\ipykernel_16896\1809277217.py:13: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  labels_mapping[i] = mode(train_data_labels[mask])[0][0]
D:\Anaconda3\lib\site-packages\scipy\stats\_stats_py.py:110: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  warnings.warn("The input array could not be properly "
C:\Users\Edgar\AppData\Local\Temp\ipykernel_16896\1809277217.py:13: DeprecationWarning: Support for non-numeric arrays has been deprecated as of SciPy 1.9.0 and will be removed in 1.11.0. `pandas.DataFrame.mode` can be used instead, see h

##  6. 分析和可视化

In [7]:
## 请使用matplotlib等可视化库对你的实验结果进行可视化分析。
## 包括但不限于准确率的对比、错误分类的分析、特征的影响等。


## 7. 结果讨论
讨论你的模型性能，尝试解释为什么某些模型比其他模型表现好，以及可能的改进方法。

## 8. 保存模型（可选）
如果需要，可以在这里添加代码保存你的模型。